<a href="https://colab.research.google.com/github/sumaaithal/PySpark_30Days_Challenge/blob/main/pyspark20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz
!tar xf spark-3.5.0-bin-hadoop3.tgz
!pip install -q findspark

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:4 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease [18.1 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:10 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [48.6 kB]
Hit:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:12 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy/main Sources [2,256 kB]
Get:13 https://ppa.launchpadcontent.net/c2d4

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.0-bin-hadoop3"

In [3]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # Property used to format output tables better
spark

In [4]:
df = spark.read.option("header",True).csv("/content/simple-zipcodes.csv")

In [5]:
df.printSchema()

root
 |-- RecordNumber: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Zipcode: string (nullable = true)
 |-- State: string (nullable = true)



In [7]:
df.write.option("header",True)\
  .partitionBy('State')\
  .mode("overwrite")\
  .csv("/tmp/zipcode-state")

In [20]:
df.write.option("header",True)\
  .partitionBy('State','City')\
  .mode("overwrite")\
  .csv("/tmp/zipcode-state-city")

In [9]:
df.write.option("header",True)\
  .option("maxRecordsPerFile",2)\
  .partitionBy('State')\
  .mode("overwrite")\
  .csv("/tmp/zipcode-state")

In [11]:
!ls /tmp/zipcode-state/State=AL

part-00000-674df45e-647d-40bb-93c3-0476135b6683.c000.csv
part-00000-674df45e-647d-40bb-93c3-0476135b6683.c001.csv


In [12]:
singleDF = spark.read.option("header",True)\
                .csv("/tmp/zipcode-state/State=AL")

In [14]:
singleDF.printSchema()

root
 |-- RecordNumber: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Zipcode: string (nullable = true)



In [15]:
singleDF.show()

+------------+-------+-------------+-------+
|RecordNumber|Country|         City|Zipcode|
+------------+-------+-------------+-------+
|       54356|     US|  SPRUCE PINE|  35585|
|       54354|     US|SPRING GARDEN|  36275|
|       54355|     US|  SPRINGVILLE|  35146|
+------------+-------+-------------+-------+



In [16]:
singleDF.createOrReplaceTempView("zipcode")

In [19]:
spark.sql("select * from zipcode where City='SPRINGVILLE'").show()

+------------+-------+-----------+-------+
|RecordNumber|Country|       City|Zipcode|
+------------+-------+-----------+-------+
|       54355|     US|SPRINGVILLE|  35146|
+------------+-------+-----------+-------+



In [31]:
!ls /tmp/zipcode-state-city/State=AL/City=SPRINGVILLE

part-00000-25c94451-1344-4aa3-ad17-babee078ead0.c000.csv


In [32]:
singleDF1 = spark.read.option("header",True)\
                .csv("/tmp/zipcode-state-city/State=AL/City=SPRINGVILLE")

In [33]:
singleDF1.show()

+------------+-------+-------+
|RecordNumber|Country|Zipcode|
+------------+-------+-------+
|       54355|     US|  35146|
+------------+-------+-------+



In [34]:
singleDF1.createOrReplaceTempView("zipcodecitystate")

In [35]:
spark.sql("select * from zipcodecitystate").show()

+------------+-------+-------+
|RecordNumber|Country|Zipcode|
+------------+-------+-------+
|       54355|     US|  35146|
+------------+-------+-------+

